# Heterogenous treatment effect models
Julian Hsu
30 Aug 2021

### Table of Contents with Navigation Links
* [Write ML Models](#Section1)
* [Simulator Functions](#Section2)
* [Many Simulations](#Section3)

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np
import os as os 
import scipy.stats 

from matplotlib import gridspec
import matplotlib.pyplot as plt
%matplotlib inline  

import statsmodels.api as sm
from IPython.display import display    

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, LinearRegression, Lasso, LassoCV, Ridge
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import mean_squared_error

import warnings



In [7]:
os.chdir('/Users/hsujulia/Amazon WorkDocs Drive/My Documents/Python')

In [6]:
print(os.getcwd())


/Users/hsujulia/Amazon WorkDocs Drive/My Documents/Python


In [8]:
# print(os.getcwd())

import HTELibrary as hte

OSError: [Errno 5] Input/output error: '/Users/hsujulia/Amazon WorkDocs Drive/My Documents/Python'

In [ ]:
def gen_data_hte(N, hte_type):
    '''
    Generate data with different types of HTE forms:
    '''
    
    df = pd.DataFrame()
    x = np.random.uniform(0,1, N)
    y = 10 + 2*(np.log(1+x)) + x**2 + np.random.normal(0,1,N)
    
    if hte_type=='constant':
        hte = 1
    elif hte_type=='linear':
        hte = 2*x
    elif hte_type=='quadratic':
        hte = 0.5*x**2
    elif hte_type=='piecewise':
        hte = np.zeros_like(x)
        hte[(x < 0.20)] = 1
        hte[(x >=0.20) & (x < 0.80) ] = 2
        hte[(x >=0.80) ] = 5
    elif hte_type=='log':
        hte = np.log(1+x*2)*1.5
    elif hte_type=='piecewise_linear':
        hte = np.zeros_like(x)
        hte[(x < 0.20)] = x[(x < 0.20)]*0.1
        hte[(x >=0.20) & (x < 0.80) ] = x[(x >=0.20) & (x < 0.80) ]*0.75
        hte[(x >=0.80) ] = x[(x >=0.80) ]*(-0.5)
    elif hte_type=='trig':
        hte = np.sin(1+x*10)*x
    else:
        print('Choose a hte type!')
        
    t = ( np.exp(x) / (1+np.exp(x)) > np.random.uniform(0,1,N)).astype(float)
    y  += t*hte
    return pd.DataFrame(data={'x':x, 't':t, 'y':y, 'hte': hte})

In [ ]:
model_max_iter = 5000
## treatment prediction models
t_models = {}
t_models['LogitCV'] = LogisticRegressionCV(cv=5, random_state=27, n_jobs=-1)
t_models['logit'] = LogisticRegression(penalty='l2',solver='lbfgs', C=1, max_iter=model_max_iter, fit_intercept=True)
t_models['logit_L1_C2'] = LogisticRegression(penalty='l1',C=2, max_iter=model_max_iter, fit_intercept=True)
t_models['logit_L2_C5'] = LogisticRegression(penalty='l2',C=2, max_iter=model_max_iter, fit_intercept=True)
t_models['rf_md10'] = RandomForestClassifier(n_estimators=25,max_depth=10, min_samples_split=200,n_jobs=-1)
t_models['rf_md3'] = RandomForestClassifier(n_estimators=25,max_depth=3, min_samples_split=200,n_jobs=-1)
t_models['nn'] = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 2), random_state=1,max_iter=model_max_iter)
## outcome prediction models
y_models = {}
y_models['LassoCV'] = LassoCV(cv=5, n_jobs=-1, normalize=True, random_state=27)
y_models['ols'] = LinearRegression()
y_models['lasso_a2'] = Lasso(alpha=2,max_iter=model_max_iter)
y_models['ridge_a2'] = Ridge(alpha=2,max_iter=model_max_iter)
y_models['rf_md10'] = RandomForestRegressor(n_estimators=25,max_depth=10, min_samples_split=200,n_jobs=-1)
y_models['rf_md3'] = RandomForestRegressor(n_estimators=25,max_depth=3, min_samples_split=200,n_jobs=-1)
y_models['nn'] = MLPRegressor(alpha=1e-5, hidden_layer_sizes=(3, 2), random_state=1, max_iter=model_max_iter)

In [ ]:
aux_dictionary = {'force_second_stage':'CVLasso'}
n_data_splits = 5
lasso_max_iter = 5000
lasso_alpha = 5

from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

In [ ]:
N = 5000
i = 0
for t in ['constant','linear','quadratic','piecewise','piecewise_linear']:

    df = gen_data_hte(N, t)
    
    df['x2'] = df['x'].pow(2)
    df['x2_ln'] = np.log(1+df['x'])    
#     for p in range(9):
#         df['x_'+str(p)] = ( np.round(df['x'],1)==np.round(p*0.1,1) ).astype(float)
#         df['x_X'+str(p)] = df['x_'+str(p)]*df['x']
    full_feature_list = [ x for x in df.columns if 'x' in x]

    ## 3. Run through each combination of models
    SGCT = hte.het_dml_approaches.SGCT(df, 
                    full_feature_list, 'y', 't',
                    full_feature_list,
                    y_models['ols'], t_models['logit'],
                   n_data_splits, aux_dictionary)
    df['dml_SGCT_TE'] = SGCT[0]
    df['dml_SGCT_SE'] = SGCT[1]    
    df['dml_SGCT_P'] = SGCT[2]
    df['dml_SGCT_Y0'] = SGCT[3]        

    HR = hte.het_dml_approaches.HR(df, 
                    full_feature_list, 'y', 't',
                    full_feature_list,
                    y_models['ols'], t_models['logit'],
                   n_data_splits, aux_dictionary)
    df['dml_HR_TE'] = HR[0]
    df['dml_HR_SE'] = HR[1]    
    df['dml_HR_P'] = HR[2]
    df['dml_HR_Y0'] = HR[3]

    dr_cvl = hte.other.DR(df, 
                    full_feature_list, 'y', 't',
                    full_feature_list,
                    y_models['ols'], t_models['logit'],
                   n_data_splits, {'force_second_stage':'CVLasso'})
    df['DR_CVL_TE'] = dr_cvl[0]
    df['DR_CVL_SE'] = dr_cvl[1]   
    df['DR_CVL_P'] = dr_cvl[2]
    df['DR_CVL_Y0'] = dr_cvl[3]


    tree_grf = hte.trees.grf(df, 
                    full_feature_list, 'y', 't',
                    full_feature_list,
                    y_models['ols'], t_models['logit'],
                   n_data_splits, {'n_estimators':1000})
    df['GRF_TE'] = tree_grf[0]
    df['GRF_SE'] = tree_grf[1]
    df['GRF_P'] = tree_grf[2]
    df['GRF_Y0'] = tree_grf[3]

    fig,ax = plt.subplots(nrows=1, ncols=2, figsize=(20,10),sharex=True, sharey=True)

    ax[0].scatter(df['x'], df['dml_SGCT_TE'], label='SGCT', alpha = 0.15, color='green')
    ax[0].scatter(df['x'], df['dml_HR_TE'], label='HR', alpha = 0.15, color='coral')
    ax[0].scatter(df['x'], df['hte'], label='Truth', alpha = 0.25, color='royalblue')

    ax[0].legend(loc='upper left', fontsize=15)
    ax[0].grid()
    ax[0].set_ylabel('HTE', fontsize=15)
    ax[0].set_xlabel('x', fontsize=15)
    
    ax[1].scatter(df['x'], df['DR_CVL_TE'], label='DoublyRobust', alpha = 0.15, color='purple')
    ax[1].scatter(df['x'], df['GRF_TE'], label='GRF', alpha = 0.15, color='pink')
    ax[1].scatter(df['x'], df['hte'], label='Truth', alpha = 0.25, color='royalblue')
    
    ax[1].legend(loc='upper left', fontsize=15)
    ax[1].grid()
    ax[1].set_ylabel('HTE', fontsize=15)
    ax[1].set_xlabel('x', fontsize=15)
    fig.subplots_adjust(wspace=0.05, hspace=0)
    fig.suptitle('Performance when Functional Form is {0}'.format(i), fontsize=15) 
    fig.patch.set_facecolor('white')
    fig.show()
    i+=1

In [ ]:
N = 5000
i = 0
for t in ['constant','linear','quadratic','piecewise','piecewise_linear']:

    df = gen_data_hte(N, t)
    
    df['x2'] = df['x'].pow(2)
    df['x2_ln'] = np.log(1+df['x'])
    for p in range(9):
        df['x_'+str(p)] = ( np.round(df['x'],1)==np.round(p*0.1,1) ).astype(float)
        df['x_X'+str(p)] = df['x_'+str(p)]*df['x']
    full_feature_list = [ x for x in df.columns if 'x' in x]

    ## 3. Run through each combination of models
    SGCT = hte.het_dml_approaches.SGCT(df, 
                    full_feature_list, 'y', 't',
                    full_feature_list,
                    y_models['ols'], t_models['logit'],
                   n_data_splits, aux_dictionary)
    df['dml_SGCT_TE'] = SGCT[0]
    df['dml_SGCT_SE'] = SGCT[1]    
    df['dml_SGCT_P'] = SGCT[2]
    df['dml_SGCT_Y0'] = SGCT[3]        

    HR = hte.het_dml_approaches.HR(df, 
                    full_feature_list, 'y', 't',
                    full_feature_list,
                    y_models['ols'], t_models['logit'],
                   n_data_splits, aux_dictionary)
    df['dml_HR_TE'] = HR[0]
    df['dml_HR_SE'] = HR[1]    
    df['dml_HR_P'] = HR[2]
    df['dml_HR_Y0'] = HR[3]

    dr_cvl = hte.other.DR(df, 
                    full_feature_list, 'y', 't',
                    full_feature_list,
                    y_models['ols'], t_models['logit'],
                   n_data_splits, {'force_second_stage':'CVLasso'})
    df['DR_CVL_TE'] = dr_cvl[0]
    df['DR_CVL_SE'] = dr_cvl[1]   
    df['DR_CVL_P'] = dr_cvl[2]
    df['DR_CVL_Y0'] = dr_cvl[3]


    tree_grf = hte.trees.grf(df, 
                    full_feature_list, 'y', 't',
                    full_feature_list,
                    y_models['ols'], t_models['logit'],
                   n_data_splits, {'n_estimators':1000})
    df['GRF_TE'] = tree_grf[0]
    df['GRF_SE'] = tree_grf[1]
    df['GRF_P'] = tree_grf[2]
    df['GRF_Y0'] = tree_grf[3]

    fig,ax = plt.subplots(nrows=1, ncols=2, figsize=(20,10),sharex=True, sharey=True)

    ax[0].scatter(df['x'], df['dml_SGCT_TE'], label='SGCT', alpha = 0.15, color='green')
    ax[0].scatter(df['x'], df['dml_HR_TE'], label='HR', alpha = 0.15, color='coral')
    ax[0].scatter(df['x'], df['hte'], label='Truth', alpha = 0.50, color='royalblue')

    ax[0].legend(loc='upper left', fontsize=20)
    ax[0].grid()
    ax[0].set_ylabel('HTE', fontsize=20)
    ax[0].set_xlabel('x', fontsize=20)
    ax[0].tick_params(axis='both', which='major', labelsize=20)
    
    ax[1].scatter(df['x'], df['DR_CVL_TE'], label='Proxy', alpha = 0.15, color='purple')
    ax[1].scatter(df['x'], df['GRF_TE'], label='GRF', alpha = 0.25, color='pink')
    ax[1].scatter(df['x'], df['hte'], label='Truth', alpha = 0.50, color='royalblue')
    
    ax[1].legend(loc='upper left', fontsize=20)
    ax[1].grid()
    ax[1].set_ylabel('HTE', fontsize=20)
    ax[1].set_xlabel('x', fontsize=20)
    ax[1].tick_params(axis='both', which='major', labelsize=20)
    fig.subplots_adjust(wspace=0.05, hspace=0)
    fig.suptitle('Performance when Functional Form is {0}'.format(i), fontsize=20) 
    fig.patch.set_facecolor('white')
    fig.show()
    i+=1